In [237]:
from ipstartup import *
import geopandas as gpd
from marley import config as c
import rasterio
from rasterio import features
from shapely import wkt
from shapely.geometry import shape, Point
from geopy.distance import distance, geodesic
deg2km = geodesic((5,3), (6,3)).km
km2deg = 1/deg2km
deg2km, km2deg
repo = r"C:\Users\simon\OneDrive\Documents\py\live\renewable"
os.chdir(c.localpath)
os.chdir("clusters_500m")

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-24 17:12)


time: 32 ms


In [239]:
with rasterio.open("../grid3/lights.tif") as src:
    lights = src.read(1)
mask = lights>0
shapes = features.shapes(lights, mask=mask)
shapes = [s[0] for s in shapes]

transform = rasterio.open("../grid3/lights.tif").transform
lonlat_shapes = []
for s in shapes:
    coords = [(x,y)*transform for x,y in s["coordinates"][0]]
    lonlat_shapes.append(Polygon(coords))

gdf = gpd.GeoDataFrame.from_dict(dict(geometry=lonlat_shapes))
gdf.to_file("../grid3/lights.geojson", driver="GeoJSON", encoding="utf8")

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-24 17:13)
[fiona._env:ERROR]:../grid3/lights.geojson: No such file or directory (collection.py:157, time=Dec-24 17:13)
[fiona._env:WARNING]:driver GeoJSON does not support creation option ENCODING (collection.py:157, time=Dec-24 17:13)


time: 8.95 s


In [ ]:
get_latlon(unlit, f.transform, inplace=True)

In [4]:
# read data
df = pd.read_excel("clusters.xlsx")
df.geometry = df.geometry.apply(wkt.loads)
df.centroid = df.centroid.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df)

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-24 12:04)


time: 1.71 s


# add data

In [8]:
# ongrid
grid = gpd.read_file(r"../nigeria-electricity-transmission-network\Nigeria Electricity Transmission Network.shp")
ongrid = gpd.GeoSeries(grid.buffer(15*km2deg)).unary_union
df["ongrid"] = gdf.intersects(ongrid)
gpd.GeoSeries(ongrid).to_file("ongrid15.geojson", driver="GeoJSON", encoding="utf8")

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-24 12:06)


time: 1.44 s


In [258]:
# area
areafile = f"{repo}/task2_estimate_energy_demand/nga_adm_osgof_20190417_SHP/nga_admbnda_adm2_osgof_20190417.shp"
area = gpd.read_file(areafile)[['ADM1_EN','ADM2_EN','geometry']]
adm1s = []
adm2s = []
for point in df.centroid:
    for adm1,adm2,po in area.values:
        if po.contains(point)==True:
            adm1s.append(adm1)
            adm2s.append(adm2)
            break
    else:
        adm1s.append("")
        adm2s.append("")
df["adm1"] = adm1s
df["adm2"] = adm2s

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-23 17:39)


time: 5.12 s


# score

In [232]:
# set score
df["score"] = df.density
df.loc[df.ongrid, "score"] = 0
##### add other criteria

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-23 17:21)


time: 25 ms


In [233]:
# output
df = df.sort_values("score", ascending=False)
df.to_excel("clusters.xlsx", index=False)
df.head(10)

[cellevents:INFO]:starting (cellevents.py:43, time=Dec-23 17:21)


,label,population,area,density,address,address0,geolocator,label_out,geometry,centroid,ongrid,adm1,adm2,score
591,591,4941,0,19841,"Koya, Kano, Nigeria",Koya,Photon,"591-Koya=4,941","POLYGON ((7.95250 11.87750, 7.95250 11.88000, ...",POINT (7.954490723470158 11.88046311671374),False,Kano,Gwarzo,19841
239,239,4222,0,17878,"Dawa, Guri, Jigawa, Nigeria",Dawa,Nominatim,"239-Dawa=4,222","POLYGON ((10.41000 12.67167, 10.40917 12.67250...",POINT (10.41210645432109 12.67365755797058),False,Jigawa,Guri,17878
647,647,3769,0,17556,"Kamagata Health Post, Dosan, Kano, Nigeria",Kamagata Health Post,Photon,"647-Kamagata Health Post=3,769","POLYGON ((8.76167 11.77167, 8.76083 11.77250, ...",POINT (8.762578104477551 11.77406265417837),False,Kano,Dawakin Kudu,17556
245,245,9434,0,16520,"Marma, Jigawa, Nigeria",Marma,Photon,"245-Marma=9,434","POLYGON ((10.33917 12.65333, 10.33833 12.65500...",POINT (10.34233549943618 12.65689708045506),False,Jigawa,Kiri Kasamma,16520
430,430,4638,0,16367,"Jirgabawa Health Post, Kunya to Niger Border R...",Jirgabawa Health Post,Photon,"430-Jirgabawa Health Post=4,638","POLYGON ((8.54333 12.19750, 8.54250 12.19833, ...",POINT (8.544662678767171 12.20011920008523),False,Kano,Minjibir,16367
724,724,6349,0,15900,"Hamdullahi, Kano, Nigeria",Hamdullahi,Photon,"724-Hamdullahi=6,349","POLYGON ((9.06083 11.62667, 9.05750 11.62750, ...",POINT (9.059060584345076 11.62959106384988),False,Kano,Albasu,15900
770,770,5381,0,15864,"Tukur, Jigawa, Nigeria",Tukur,Photon,"770-Tukur=5,381","POLYGON ((9.76583 11.53750, 9.76500 11.53833, ...",POINT (9.767447892100485 11.54017376622942),False,Jigawa,Buji,15864
314,314,4268,0,15778,"Tsaraka Health Post, Gwarabjawa, Kano, Nigeria",Tsaraka Health Post,Photon,"314-Tsaraka Health Post=4,268","POLYGON ((8.60667 12.43750, 8.60500 12.43833, ...",POINT (8.606187480106891 12.44085431816539),False,Kano,Dambatta,15778
850,850,5235,0,15631,"Maruta, Gwaram, Jigawa, Nigeria",Maruta,Nominatim,"850-Maruta=5,235","POLYGON ((9.86333 11.37417, 9.86250 11.37500, ...",POINT (9.862801393489818 11.37833348910203),False,Jigawa,Gwaram,15631
721,721,7968,0,15211,"Lafara, Jigawa, Nigeria",Lafara,Photon,"721-Lafara=7,968","POLYGON ((9.62917 11.63250, 9.62583 11.63417, ...",POINT (9.630446835888513 11.63561609676373),False,Jigawa,Birni Kudu,15211


time: 1.39 s
